In [ ]:
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_cats.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt

!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_aspects.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_reviews.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_cats.txt

!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_cats.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt




In [ ]:
!pip install pymorphy2
!pip install transformers
!pip install pytextspan

In [3]:
import pandas as pd
import numpy as np
from textspan import get_original_spans 
import nltk
from nltk.collocations import *
from nltk.metrics.association import QuadgramAssocMeasures

import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize
from collections import defaultdict, Counter

from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel

from string import punctuation

tokenizer_bert = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model_bert = AutoModel.from_pretrained("cointegrated/rubert-tiny")
#model_bert.cuda()  


morph = pymorphy2.MorphAnalyzer() 


Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Загрузка и предобработка

In [4]:
train_asp = pd.read_csv(
    'train_split_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv('train_split_reviews.txt', delimiter='\t', names=['text_id','text'])
dev_texts = pd.read_csv('dev_reviews.txt', delimiter='\t', names=['text_id', 'text'])
dev_asp = pd.read_csv(
    'dev_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)

In [5]:
def tokenize(text):
    tokens = [token.lower() for token in simple_word_tokenize(text)]
    return tokens


def normalize(text):
    tokens = [token.lower() for token in simple_word_tokenize(text)]
    text = [morph.parse(token)[0].normal_form for token in tokens]
    return tuple(text)


def pos_tag(mention):
    POS_tag_list = []
    for m in mention:
        p = morph.parse(m)[0]
        POS_tag_list.append(p.tag.POS )
    return tuple(POS_tag_list)

In [6]:
train_asp['norm_mention'] = [normalize(m) for m in train_asp['mention']]
train_asp['length'] = train_asp['norm_mention'].apply(len)
train_asp['pos_tag'] = train_asp['norm_mention'].apply(pos_tag)

dev_texts['tokens'] = dev_texts['text'].apply(tokenize)
dev_texts['norm_mention'] = dev_texts['text'].apply(normalize)
dev_texts['normalized_text'] = dev_texts['norm_mention'].apply(' '.join)
dev_texts['length'] = dev_texts['norm_mention'].apply(len)

dev_asp['norm_mention'] = dev_asp['mention'].apply(normalize)
dev_asp['length'] = dev_asp['norm_mention'].apply(len)
dev_asp['pos_tag'] = dev_asp['norm_mention'].apply(pos_tag)


In [7]:
not_in_train  = set(dev_asp['pos_tag']) - set(train_asp['pos_tag']) #схемы, которых нет в трэин аспектах, но они есть в тестовых
pos_tag_cnt = Counter(dev_asp['pos_tag'].to_list()) # сделаем каунтер для тестовых схем: сколько встречается каждая

for scheme in not_in_train:
    print(scheme, ':', pos_tag_cnt[scheme])   #вот такие схемы в таком количестве мы выкинем фильтрация по постэг схемам трэин аспектов

('NOUN', 'ADJF', None, 'ADJF', None) : 1
('ADJS', 'PREP', 'NOUN') : 1
('NOUN', 'PREP', 'ADVB', None, 'PREP', 'NOUN', None) : 1
('PREP', 'INFN') : 1
('ADJF', 'NOUN', 'PREP', 'NOUN', 'PREP', 'ADJF') : 1
('NOUN', 'GRND') : 1
(None, None, None) : 1
('INFN', 'NPRO', 'NOUN') : 1
('NOUN', 'PREP', 'NUMR', 'NOUN') : 1
('INFN', None, 'NOUN') : 1
('ADJF', 'NOUN', 'ADJF', 'NOUN') : 1
('NOUN', 'CONJ', 'NOUN', 'INFN') : 1
('NOUN', 'PREP', 'NOUN', None, 'NOUN', 'CONJ', 'NOUN') : 1
('NOUN', 'CONJ', 'NOUN', 'NOUN') : 1
('ADJF', 'PREP', 'NOUN') : 1
('NOUN', 'NOUN', 'PREP', 'ADJF', 'NOUN') : 1
('ADJF', 'NOUN', 'CONJ', 'NOUN') : 1
('NOUN', 'NOUN', 'PREP', 'ADJF', 'NOUN', 'CONJ', 'ADJF', 'NOUN') : 1
('ADJF', None, 'ADJF', None, 'ADJF', 'NOUN') : 1
('INFN', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'NOUN', 'PREP', 'ADJF', 'NOUN', 'CONJ', 'ADJF', 'NOUN') : 1
('NOUN', None, None, None) : 2
('NOUN', 'PREP', 'ADJF', 'ADJF', 'NOUN') : 1


## Функции

### функции для вычислений

In [8]:
def count_similarity(sim_matrix, train_asp_indexed):
    similarity_vec = cosine_similarity(sim_matrix, train_asp_indexed)
    return similarity_vec


def index_bert(docs):
    doc_vecs = []
    if type(docs) != list:
        docs = [docs] 
    for doc in docs:
        t = tokenizer_bert(doc, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = model_bert(**{k: v.to(model_bert.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        doc_vecs.append(embeddings[0].cpu().numpy())
    return np.vstack(doc_vecs) 


def index_ngrams(ngrams):
    ngrams = [' '.join(ngram) for ngram in ngrams]
    indexed = index_bert(ngrams)
    return indexed


def filter_by_cossim(similarity_vec, ngram_found, threshold):
    sim_vec = np.squeeze(similarity_vec)
    order = np.array(np.squeeze(sim_vec) > threshold)
    return ngram_found[order], sim_vec[order]

###Функция для выделения ближайших нграмм

In [9]:
def find_similar_ngrams(train_asp, n, threshold, dev_texts=None, measures=None, finder=None, topn_assoc=None): #n - ngrams, topn_assoc - сколько берем по ассоциативной метрике, 
    train_list = train_asp[train_asp.length == n].norm_mention.tolist() #список нграмм из трэин аспектов

    if n == 1:
        n_grams = []
        for text_tokens in dev_texts.norm_mention:
            n_grams.extend(tuple([token]) for token in text_tokens)

    else:
        n_grams = finder.nbest(measures.jaccard, topn_assoc)  #находим нграммы по ассоциативной метрике из тестовых текстов
  
    filter_n_grams = [ngram for ngram in n_grams if pos_tag(ngram) in set(train_asp['pos_tag'])] #фильтруем по postag схеме

    found_ngrams_indexed = index_ngrams(filter_n_grams) #индексируем бертом все найденные нграммы возвращает матрицу размера количество биграм*длину эмбеддингов

    top_n_similar = []
    one_list = []
    values = {}

    for ngram in train_list: #идем по аспектным биграммам из трэина

        ngram_indexed = index_ngrams([ngram])  #индексируем их - возвращает матрицу размера количество биграм*длину эмбеддингов
        rez = count_similarity(found_ngrams_indexed, ngram_indexed) #для биграммы из теста считаем косинусную близость с каждой биграммой из трэин аспектов
        rez = np.squeeze(rez, axis=1)  #избавляемся от лишних размерностей
        rez = np.expand_dims(rez, axis=0) #добавляем размерность в нужном месте для сортировки
        toprez_ngrams, toprez_values = filter_by_cossim(rez, np.array(list(filter_n_grams)), threshold) #сортируем по значению и берем топн ближайших ключей

        toprez_list = list(map(tuple, toprez_ngrams))  #убираем размерности и делаем лист из тюплов
        top_n_similar.append(toprez_list)
        one_list.extend(toprez_list)
        

        for one, rez in zip(toprez_list, toprez_values): #сделаем словарь: нграмма - максимальная близость к трэин аспектам
            if one in values:
               if rez > values[one][0]:
                    values[one] = [rez, ngram]
            else:
                values[one] = [rez, ngram]

    return values, one_list

###функция для выделения категорий

In [10]:
def get_mention_category(data, cat_type, values):
    nrgam2cat = {}
    mention_categories = data.value_counts(subset=['norm_mention', cat_type])
    mention_categories_dict = defaultdict(dict)
    for key, value in mention_categories.items():
        mention_categories_dict[key[0]][key[1]] = value
    most_commons = {k: Counter(v).most_common(1)[0][0] for k, v in mention_categories_dict.items()}
    for ngram in values:  #выделенной нграмме
        category_definer = values[ngram][1]
        nrgam2cat[ngram] = most_commons[category_definer]
  
    return nrgam2cat

##Униграммы


In [11]:
values1, one_list1 = find_similar_ngrams(train_asp, 1, 0.999, dev_texts=dev_texts)
value_cat1 = get_mention_category(train_asp, 'category', values1)

In [12]:
len(value_cat1)

266

## Биграммы

In [13]:
# Коллекция метрик
bigram_measures = nltk.collocations.BigramAssocMeasures()

ignored = punctuation

# Экстрактор коллокаций
finder2 = BigramCollocationFinder.from_documents(dev_texts.norm_mention.tolist())
finder2.apply_word_filter(lambda w: w.lower() in ignored)


In [14]:
values2, one_list2 = find_similar_ngrams(train_asp, 2, 0.988, measures=bigram_measures, finder=finder2, topn_assoc=6000)
value_cat2 = get_mention_category(train_asp, 'category', values2)

In [15]:
len(values2)

64

##Триграммы

In [16]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()

finder3 = TrigramCollocationFinder.from_documents(dev_texts.norm_mention.tolist())
finder3.apply_word_filter(lambda w: w.lower() in ignored)

In [17]:
values3, one_list3 = find_similar_ngrams(train_asp, 3, 0.989, measures=trigram_measures, finder=finder3, topn_assoc=8000)
value_cat3 = get_mention_category(train_asp, 'category', values3)

In [18]:
len(values3)

12

##4-граммы


In [19]:
quadgram_measures = QuadgramAssocMeasures()

finder4 = QuadgramCollocationFinder.from_documents(dev_texts.norm_mention.tolist())
finder4.apply_word_filter(lambda w: w.lower() in ignored)

In [20]:
values4, one_list4 = find_similar_ngrams(train_asp, 4, 0.97, measures=quadgram_measures, finder=finder4, topn_assoc=3980)
value_cat4 = get_mention_category(train_asp, 'category', values4)

In [21]:
len(values4)

4

## Ищем выделенные n-граммы и формируем итоговый файл

In [22]:
def get_token_spans(row):
  spans = [span[0] for span in get_original_spans(row['tokens'], row['text'])]
  return spans


def ngram_index(words, ngram, offset):
  try:
    ind_start = offset + list(nltk.ngrams(words[offset:], len(ngram))).index(tuple(ngram))
    ind_end = ind_start + len(ngram) - 1
    return (ind_start, ind_end)
  except ValueError:
    return None

def find_text_ngrams(norm_text, ngrams, spans):
  ngrams_text = []

  for ngram in ngrams:
    offset = 0
    i = 0
    while True and len(norm_text) != offset:
      ngram_ind = ngram_index(norm_text, ngram, offset)
      if not ngram_ind:
        break

      if ngram_ind:
        first_token_ind = ngram_ind[0]
        last_token_ind = ngram_ind[-1]

        start = spans[first_token_ind][0]
        end = spans[last_token_ind][-1]

        offset = last_token_ind + 1

        ngrams_text.append((ngram, start, end))

  return ngrams_text


In [23]:
dev_texts['spans'] = dev_texts.apply(lambda x: get_token_spans(x), axis=1)
total = {**value_cat4, **value_cat2, **value_cat3, **value_cat1}
len(total)

346

In [24]:
with open('dev_pred_aspects.txt', 'w') as f:
    for tokens, span, id, text in zip(dev_texts.norm_mention.values, dev_texts.spans.values, dev_texts.text_id.values, dev_texts.text):
        ngrams_spans = find_text_ngrams(tokens, total.keys(), span)
        for ngram, start, end in ngrams_spans:
            print(id, total[ngram], text[start:end], start, end, 'neutral', sep="\t", file=f)